<a href="https://colab.research.google.com/github/lekshmilekshmi/ICT_DSA_ACTIVITIES/blob/main/lekshmi_ETL_Casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)

In [ ]:
print("Number of records:", len(df))
print("Number of unique locations:", df['place'].nunique())
print("\nTop 5 rows:\n", df.head())
print("\nColumn names:\n", df.columns.tolist())

Number of records: 9906
Number of unique locations: 5250

Top 5 rows:
                        time   latitude   longitude  depth   mag magType   nst  \
0  2025-09-21T14:09:14.470Z  33.936333 -116.677167  14.43  1.24      ml  70.0   
1  2025-09-21T13:56:53.920Z  35.393500 -117.748667   5.91  0.97      ml  38.0   
2  2025-09-21T13:56:33.600Z  38.769333 -122.730164   2.12  1.13      md   9.0   
3  2025-09-21T13:50:53.610Z  33.031500 -116.292333   6.80  0.33      ml  20.0   
4  2025-09-21T13:46:15.460Z  35.638833 -117.456000   7.07  0.95      ml  27.0   

    gap     dmin   rms  ...                   updated  \
0  22.0  0.08250  0.16  ...  2025-09-21T14:13:32.763Z   
1  62.0  0.10160  0.13  ...  2025-09-21T14:13:41.254Z   
2  64.0  0.00877  0.01  ...  2025-09-21T13:58:09.704Z   
3  56.0  0.07725  0.20  ...  2025-09-21T13:54:24.380Z   
4  54.0  0.03714  0.14  ...  2025-09-21T14:15:28.660Z   

                             place        type horizontalError depthError  \
0         10 km ENE of

In [ ]:
# Convert time to datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [ ]:
# Drop missing latitude, longitude, mag
df = df.dropna(subset=['latitude', 'longitude', 'mag'])

In [ ]:
# Filter magnitude >= 4.0
df = df[df['mag'] >= 4.0]

In [ ]:
# Add day_of_week column
df['day_of_week'] = df['time'].dt.day_name()


In [ ]:
# Add severity_level column
def classify_severity(mag):
    if mag < 4.0:
        return "Low"
    elif 4.0 <= mag < 6.0:
        return "Moderate"
    else:
        return "High"

df['severity_level'] = df['mag'].apply(classify_severity)


In [ ]:
# Count earthquakes per place
eq_per_place = df['place'].value_counts().reset_index()
eq_per_place.columns = ['place', 'earthquake_count']
print("\nTop 10 locations with most earthquakes:\n", eq_per_place.head(10))


Top 10 locations with most earthquakes:
                                             place  earthquake_count
0                   South Sandwich Islands region                13
1                                   Drake Passage                 9
2                       south of the Fiji Islands                 7
3                                     Fiji region                 6
4                       Izu Islands, Japan region                 6
5  146 km ESE of Petropavlovsk-Kamchatsky, Russia                 4
6  153 km ESE of Petropavlovsk-Kamchatsky, Russia                 4
7                southeast of the Loyalty Islands                 4
8                                  south of Tonga                 4
9    150 km E of Petropavlovsk-Kamchatsky, Russia                 4


In [ ]:
# Compute avg magnitude and max depth per day
daily_summary = df.groupby(df['time'].dt.date).agg(
    avg_magnitude=('mag', 'mean'),
    max_depth=('depth', 'max')
).reset_index()
daily_summary.rename(columns={'time': 'date'}, inplace=True)
print("\nSummary (first 5 rows):\n",daily_summary.head())


Summary (first 5 rows):
          date  avg_magnitude  max_depth
0  2025-08-22       4.485714    515.056
1  2025-08-23       4.582979    565.087
2  2025-08-24       4.514894    499.800
3  2025-08-25       4.677778    126.766
4  2025-08-26       4.678571    612.812


In [ ]:
# Save cleaned dataset
df.to_csv("cleaned_earthquakes.csv", index=False)


In [ ]:
# Save summary dataset
daily_summary.to_csv("earthquake_summary.csv", index=False)

In [ ]:
# Store into SQLite
conn = sqlite3.connect("earthquakes.db")
df.to_sql("earthquakes_cleaned", conn, if_exists="replace", index=False)
daily_summary.to_sql("earthquake_summary", conn, if_exists="replace", index=False)
conn.close()

print("\n ETL process completed!")
print("Files saved: cleaned_earthquakes.csv, earthquake_summary.csv, earthquakes.db")


 ETL process completed!
Files saved: cleaned_earthquakes.csv, earthquake_summary.csv, earthquakes.db


#F**low chart of ETL logic**

Extract CSV → Load DataFrame → Clean Data (drop NA, filter magnitude) → Transform (datetime, severity, day_of_week) → Aggregate (per place & per day) → Save (CSV + SQLite)

#**A short explanation of process**
**Extract**: The raw data was loaded from the USGS API (CSV format) into a Pandas DataFrame. Basic dataset information such as record count, unique locations, and sample rows was examined.

**Transform**: Data was cleaned and enriched by:

Converting the time field into proper datetime format.

Removing records with missing latitude, longitude, or magnitude values.

Filtering out earthquakes with magnitude below 4.0.

Adding new fields like day_of_week and severity_level (based on magnitude ranges).

Aggregating data to count earthquakes per location and calculate daily statistics (average magnitude, maximum depth).

**Load:** The cleaned dataset and the daily summary were saved into CSV files and also stored in a SQLite database for further querying and analysis.

#**Insight from the Data**

From the processed earthquake data, we observed that the majority of recorded earthquakes had magnitudes between 4.0 and 6.0 (classified as “Moderate”), while only a small fraction exceeded magnitude 6.0. This indicates that although high-severity earthquakes do occur, they are relatively rare compared to moderate ones